# O Jogo Robo Twitter

A ideia do jogo é bastante simples:
 - O jogo inicia com uma matriz 10x10 com 12 presentes espalhados randomicamente.
 - Toda a vez que alguem digitar "tianoklein" no twitter, vai indicar que ele quer participar do jogo.
 - Existe 4 comandos: UP, DOWN, LEFT, RIGHT.
 - Deve digitar "tianoklein  comando" para andar uma posição.


## Bugs
 - Problema na hora de marcar "@usuário" para quem achou a recompensa
 - O twitter não permite 2 postagens iguais, em um curto espaço de tempo.
 - ...

## Import das Bibliotecas

In [1]:
#!pip install tweepy

In [2]:
import logging
import time
import csv
import json
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from datetime import datetime
from dateutil import parser

# Robo
Define o comportamento do Robo

In [3]:
## SUPER CLASSE: CLASSE MÃE
class Point(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return '<%s, %s>' % (self.x, self.y)

## HERDA DA MÃE (Point)
class Reward(Point):
    
    def __init__(self, x, y, name):
            #super recebe a classe, a instancia, e retorna os metodos da mãe
            super(Reward, self).__init__(x, y)
            self.name = name
            
    def __str__(self):
        return '<%s, %s>: %s' % (self.x, self.y, self.name)
    
    def __repr__(self):
        return 'Rewards %s' % str(self)
    
class Robot(Point):
    
    def move_up(self):
        if self.y < 10:
            self.y = self.y + 1
        else:
            print("movimento proibido")
            api.update_status('movimento UP proibido!')
            
    def move_down(self):
        if self.y > 0:
            self.y = self.y - 1
        else:
            print("movimento proibido")
            api.update_status('movimento DOWN proibido!')
            
    def move_left(self):
        if self.x > 0:
            self.x = self.x - 1 
        else:
            print("movimento proibido")
            api.update_status('movimento LEFT proibido!')
            
    def move_right(self):
        if self.x < 10:
            self.x = self.x + 1
        else:
            print("movimento proibido")
            api.update_status('movimento RIGHT proibido!')

In [21]:
def check_reward(robot, reward, username):
    ok=False
    for reward in rewards:
        if reward.x == robot.x and reward.y == robot.y:
            print("Parabens ooo  %s" % username)
            print("o robo acho a ooo %s" % reward.name)
            
            api.update_status("Parabens oo ! %s" % username)
            api.update_status("o robo acho a oo %s" % reward.name)
            ok=True
    return ok

In [5]:
import random

In [6]:
#Insere 12 presentes.
r1 = Reward(random.randint(0,10),random.randint(0,10), 'moeda' )
r2 = Reward(random.randint(0,10),random.randint(0,10), 'gasolina' )
r3 = Reward(random.randint(0,10),random.randint(0,10), 'arma' )
r4 = Reward(random.randint(0,10),random.randint(0,10), 'moeda' )
r5 = Reward(random.randint(0,10),random.randint(0,10), 'gasolina' )
r6 = Reward(random.randint(0,10),random.randint(0,10), 'arma' )
r7 = Reward(random.randint(0,10),random.randint(0,10), 'moeda' )
r8 = Reward(random.randint(0,10),random.randint(0,10), 'gasolina' )
r9 = Reward(random.randint(0,10),random.randint(0,10), 'arma' )
r10 = Reward(random.randint(0,10),random.randint(0,10), 'moeda' )
r11 = Reward(random.randint(0,10),random.randint(0,10), 'gasolina' )
r12 = Reward(random.randint(0,10),random.randint(0,10), 'arma' )


In [7]:
rewards  = [r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12]

In [8]:
rewards

[Rewards <0, 2>: moeda,
 Rewards <7, 8>: gasolina,
 Rewards <10, 8>: arma,
 Rewards <1, 0>: moeda,
 Rewards <8, 6>: gasolina,
 Rewards <6, 1>: arma,
 Rewards <1, 2>: moeda,
 Rewards <0, 4>: gasolina,
 Rewards <9, 5>: arma,
 Rewards <0, 4>: moeda,
 Rewards <2, 0>: gasolina,
 Rewards <1, 7>: arma]

In [9]:
#Coloca o robo em uma matriz 10x10
robot = Robot(random.randint(0,10),random.randint(0,10))

In [10]:
print(robot)

<0, 1>


# Twitter

In [11]:
# enable logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s', 
                    datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [12]:
# authorize the app to access Twitter on our behalf
# Go to http://apps.twitter.com and create an app.
# The consumer key and secret will be generated for you after

consumer_key="XXXXXXXXXXXXXXXXXXXXXXXXX"
consumer_secret="XXXXXXXXXXXXXXXXXXXXXXXX"
access_token="XXXXXXXXXXXXXXXXXXX" 
access_secret="XXXXXXXXXXXXXXXXXXXX"

In [14]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [15]:
# parse data
def parse_tweet(data):

    # load JSON item into a dict
    tweet = json.loads(data)


    # check if tweet is valid
    if 'user' in tweet.keys():

        # parse date    
        tweet['CREATED_AT'] = parser.parse(tweet['created_at'])

        # classify tweet type based on metadata
        if 'retweeted_status' in tweet:
            tweet['TWEET_TYPE'] = 'retweet'

        elif len(tweet['entities']['user_mentions']) > 0:
            tweet['TWEET_TYPE'] = 'mention'

        else:
            tweet['TWEET_TYPE'] = 'tweet'

        return tweet

    else:
        logger.warning("Imcomplete tweet: %s", tweet)


# extract relevant data to write to CSV
def extract_content(tweet):

    content = [tweet['user']['screen_name'], 
               tweet['CREATED_AT'].strftime('%Y-%m-%d %H:%M:%S'),
               tweet['TWEET_TYPE'],
               tweet['user']['location'],
               tweet['text'].encode('unicode_escape')]
    
# verifica o ROBO
    print(tweet['text'])
    s = tweet['text'].encode('unicode_escape')
    
    go_right = len(s.split('right'))-1
    go_left = len(s.split('left'))-1
    go_up = len(s.split('up'))-1
    go_down = len(s.split('down'))-1
    
    if go_right > 0:
        robot.move_right()
    elif go_left > 0:
        robot.move_left()
    elif go_up > 0:
        robot.move_up()
    elif go_down > 0:
        robot.move_down()
    else:
        print('Movimento inválido')
        api.update_status('Movimento inválido!')
    
    username = '@'+content[0]
    #username ='@PauloCKlein'
    #print(username)
    
    print(robot)
    api.update_status(robot)
    check_reward(robot,rewards, username)
    
    return content

In [16]:
class MyListener(StreamListener):

    def on_data(self, data):
        try:
            tweet = parse_tweet(data)
            content = extract_content(tweet)
            with open('tweets.csv', 'a') as f: 
                writer = csv.writer(f, quotechar = '"')
                writer.writerow(content)
                logger.info(content[3])

        except BaseException as e:
            logger.warning(e)

        return True

    def on_error(self, status):
        logger.warning(status)
        return True

In [22]:
def start_stream():

    while True:

        logger.warning("Twitter API Connection opened")

        try:
            twitter_stream = Stream(auth, MyListener())
            twitter_stream.filter(track=['tianoklein'])

        except Exception as e: 
            logger.warning(e)
            continue

        finally:
            logger.warning("Twitter API Connection closed")

In [23]:
if __name__ == '__main__':
    start_stream()

2018-01-26 09:59:56 WARNING <ipython-input-22-7b7030ccec9b> - start_stream: Twitter API Connection opened
2018-01-26 10:00:11 INFO binder - build_parameters: PARAMS: {'status': '<0, 1>'}


tianoklein down 4
<0, 1>


2018-01-26 10:00:11 INFO <ipython-input-16-902da51debd0> - on_data: Porto Alegre
2018-01-26 10:00:31 INFO binder - build_parameters: PARAMS: {'status': '<0, 2>'}
2018-01-26 10:00:31 WARNING <ipython-input-16-902da51debd0> - on_data: [{u'message': u'Status is a duplicate.', u'code': 187}]


tianoklein up 4
<0, 2>


2018-01-26 10:01:45 WARNING <ipython-input-22-7b7030ccec9b> - start_stream: Twitter API Connection closed


KeyboardInterrupt: 

## Abrir csv gerado com todos os tweets

In [ ]:
import os
cwd = os.getcwd()
print cwd
files = os.listdir(cwd) 
print files

In [ ]:
#os.remove("tweets.csv")
#print("File Removed!")

In [24]:
with open('tweets.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        print(row)

['tianoklein', '2018-01-26 14:39:57', 'tweet', 'Porto Alegre', 'tianoklein up 1']
['tianoklein', '2018-01-26 14:41:33', 'tweet', 'Porto Alegre', 'tianoklein down 2']
['tianoklein', '2018-01-26 14:56:34', 'tweet', 'Porto Alegre', 'tianoklein up 22']
['tianoklein', '2018-01-26 15:00:41', 'tweet', 'Porto Alegre', 'tianoklein up 55']
['tianoklein', '2018-01-26 15:00:51', 'tweet', 'Porto Alegre', 'tianoklein down 55']
['tianoklein', '2018-01-26 15:03:52', 'tweet', 'Porto Alegre', 'tianoklein up 6']
['tianoklein', '2018-01-26 15:08:45', 'tweet', 'Porto Alegre', 'tianoklein down 1']
['tianoklein', '2018-01-26 15:09:27', 'tweet', 'Porto Alegre', 'tianoklein down 2']
['tianoklein', '2018-01-26 15:10:10', 'tweet', 'Porto Alegre', 'tianoklein down 3']
['tianoklein', '2018-01-26 15:12:17', 'tweet', 'Porto Alegre', 'tianoklein up 4']
['tianoklein', '2018-01-26 15:12:37', 'tweet', 'Porto Alegre', 'tianoklein left 1']
['tianoklein', '2018-01-26 15:12:51', 'tweet', 'Porto Alegre', 'tianoklein up 5']
[